In [12]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNISTの手書き画像データを読み込む
mnist = input_data.read_data_sets("mnist/", one_hot=True)

pixels = 28 * 28
nums = 10

# プレースホルダを定義
x  = tf.placeholder(tf.float32, shape=(None, pixels), name="x")
y_ = tf.placeholder(tf.float32, shape=(None, nums), name="y_")

# 重みとバイアスを初期化する関数
def weight_variable(name, shape):
    W_init = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(W_init, name="W_"+name)
    return W
def bias_variable(name, size):
    b_init = tf.constant(0.1, shape=[size])
    b = tf.Variable(b_init, name="b_"+name)
    return b

# 畳み込みを行う関数
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
# 最大プーリングを行う関数
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
        strides=[1,2,2,1], padding='SAME')

# 畳み込み層1
with tf.name_scope('conv1') as scope:
    W_conv1 = weight_variable('conv1', [5, 5, 1, 32])
    b_conv1 = bias_variable('conv1', 32)
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# プーリング層1
with tf.name_scope('pool1') as scope:
    h_pool1 = max_pool(h_conv1)

# 畳み込み層2
with tf.name_scope('conv2') as scope:
    W_conv2 = weight_variable('conv2', [5, 5, 32, 64])
    b_conv2 = bias_variable('conv2', 64)
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

# プーリング層2
with tf.name_scope('pool2') as scope:
    h_pool2 = max_pool(h_conv2)

# 全結合レイヤー
with tf.name_scope('fully_connected') as scope:
    n = 7 * 7 * 64
    W_fc = weight_variable('fc', [n, 1024])
    b_fc = bias_variable('fc', 1024)
    h_pool2_flat = tf.reshape(h_pool2, [-1, n])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc) + b_fc)

# ドロップアウト(過剰適合)を排除
with tf.name_scope('dropout') as scope:
    keep_prob = tf.placeholder(tf.float32)
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)

# 読み出し層
with tf.name_scope('readout') as scope:
    W_fc2 = weight_variable('fc2', [1024, 10])
    b_fc2 = bias_variable('fc2', 10)
    y_conv = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2) + b_fc2)

# モデルの学習
with tf.name_scope('loss') as scope:
    cross_entoropy = -tf.reduce_sum(y_ * tf.log(y_conv))
with tf.name_scope('training') as scope:
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(cross_entoropy)

# モデルの評価
with tf.name_scope('predict') as scope:
    predict_step = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
    accuracy_step =tf.reduce_mean(tf.cast(predict_step, tf.float32))

# feed_dictの設定
def set_feed(images, labels, prob):
    return {x: images, y_:labels, keep_prob: prob}

# セッションを開始
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # TensorBoardへの書き込み準備
    tw = tf.summary.FileWriter('log_dir', graph=sess.graph)
    # テスト用のフィールドを生成
    test_fd = set_feed(mnist.test.images, mnist.test.labels, 1)
    # 訓練を開始
    for step in range(10000):
        batch = mnist.train.next_batch(50)
        fd = set_feed(batch[0], batch[1], 0.5)
        _, loss = sess.run([train_step, cross_entoropy], feed_dict=fd)
        if step % 100 == 0:
            acc = sess.run(accuracy_step, feed_dict=test_fd)
            print("step=", step, "loss=", loss, "acc=", acc)
    # 最終結果を表示
    acc = sess.run(accuracy_step, feed_dict=test_fd)
    print("正解率=", acc)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
step= 0 loss= 531.587 acc= 0.1179
step= 100 loss= 53.7318 acc= 0.8213
step= 200 loss= 32.7404 acc= 0.9019
step= 300 loss= 23.6931 acc= 0.9246
step= 400 loss= 37.4094 acc= 0.9345
step= 500 loss= 16.2145 acc= 0.9445
step= 600 loss= 20.0372 acc= 0.9491
step= 700 loss= 8.73677 acc= 0.9512
step= 800 loss= 19.2576 acc= 0.9526
step= 900 loss= 13.8845 acc= 0.9567
step= 1000 loss= 8.8835 acc= 0.9598
step= 1100 loss= 3.19575 acc= 0.965
step= 1200 loss= 7.22396 acc= 0.9642
step= 1300 loss= 2.46884 acc= 0.966
step= 1400 loss= 3.73675 acc= 0.9689
step= 1500 loss= 6.34133 acc= 0.9677
step= 1600 loss= 8.81936 acc= 0.9695
step= 1700 loss= 21.4133 acc= 0.9719
step= 1800 loss= 10.7832 acc= 0.9707
step= 1900 loss= 2.6238 acc= 0.9728
step= 2000 loss= 2.83861 acc= 0.9744
step= 2100 loss= 4.14944 acc= 0.9753
step= 2200 loss= 4.57132 acc